In [1]:
from keras.datasets import mnist  # Importação da base de dados
from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras import utils
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [2]:
'''
    Este trecho do código tem a função de para mudar a semente geradora dos
    numeros aleatórios. Será utilizado para obter os mesmos resultados ao
    fazer manualmente a divisão da base de dados durante a operação
    de validação cruzada
'''
seed = 5
np.random.seed(seed)

<p>Tratamento dos dados</p>

In [3]:
'''
    Carregar a base de dados, já com a separação pré-definida para teste e treinamento
    x -> atributos previsores
    y -> classes da base

    Obs: para a validação cruzada, apenas os valores de treinamento serão utilizadas,
    ou seja -> (xTrain, yTrain)
'''

(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
sizeXTRAIN = xTrain.shape
sizeYTRAIN = yTrain.shape
print('Shape dos atributos previsores de treinamento', sizeXTRAIN)
print('Shape das classes de treinamento', sizeYTRAIN)

'''
    A dimensionalidade de cada imagem será diminuida, ou seja, apenas uma camada RGB será utilizada 
    e o tipo dos dados são alterados para float32 a fim de aplicar a normalização futuramente
'''
trainForecasters = xTrain.reshape(xTrain.shape[0], 28, 28, 1)
trainForecasters = trainForecasters.astype('float32')
trainForecasters /= 255  # Normalização

'''
    Criação de dummy variables a fim de criar um encoder para 
    os diferentes valores e representar as diferentes classes 
'''

trainClass = utils.to_categorical(yTrain, 10)


Shape dos atributos previsores de treinamento (60000, 28, 28)
Shape das classes de treinamento (60000,)


In [4]:
'''
    A variável kfold busca controlar a validação cruzada, a variável recebe
    a função StratifiedKFold, com os seguintes parâmetros:

    * n_splits -> Quantidade de 'dobras' ou seções em que a base de dados será dividida
    * shuffle -> Define que os dados serão analizados aleatóriamente
    * random_state -> Quando 'shuffle' for setado como true, o random_state afeta a ordem dos indices
        controlando a aleatoriedade de cada dobra para cada classe
'''

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

'''
    O resultado de cada execução é inserido na lista da variável results
'''
results = []

In [7]:
'''
    O loop tem o objetivo de percorrer os dados selecionados para treinamento e teste em cada iteração,
    utilizando a variável kfold (criada préviamente para controlar a validação cruzada).

    'trainIndex' e 'testIndex' são definidos pelos ÍNDICES da base de dados que serão utilizados
    para realizar treinamento da rede neural em cada iteração. 
    Seus valores serão definidos pela divisão de kfold e serão sempre distintos, uma vez que um grupo
    será utilizado para treinamento e outro para teste.

    ???
    
'''

for trainIndex, testIndex in kfold.split(trainForecasters, np.zeros(shape = (trainClass.shape[0], 1))):
   
    '''
        Definição de uma CNN básica
    '''   
    classificador = Sequential()
    classificador.add(Conv2D(32, (3,3), activation = 'relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(units = 128, activation = 'relu'))
    classificador.add(Dense(units = 10, activation = 'softmax'))
    classificador.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    '''
        O treinamento da rede neural se dá por meio dos dados definidos pelo index definidos em kfold
    '''
    classificador.fit(trainForecasters[trainIndex], trainClass[trainIndex], 
                      batch_size = 128, epochs = 5)
    
    '''
        O método evaluate retorna a acurácia do treinamento em cada iteração, tais resultados são armazenados
        na lista 'resultados'
    '''
    precision = classificador.evaluate(trainForecasters[testIndex], trainClass[testIndex])

    results.append(precision[1])


                                         

Epoch 1/5
375/375 [==============================] - 7s 16ms/step - loss: 0.2440 - accuracy: 0.9298
Epoch 2/5
375/375 [==============================] - 6s 17ms/step - loss: 0.0783 - accuracy: 0.9769
Epoch 3/5
375/375 [==============================] - 6s 17ms/step - loss: 0.0499 - accuracy: 0.9849
Epoch 4/5
375/375 [==============================] - 6s 16ms/step - loss: 0.0373 - accuracy: 0.9890
Epoch 5/5
375/375 [==============================] - 1s 2ms/step - loss: 0.0549 - accuracy: 0.9837
Epoch 1/5
375/375 [==============================] - 6s 15ms/step - loss: 0.2540 - accuracy: 0.9262
Epoch 2/5
375/375 [==============================] - 5s 14ms/step - loss: 0.0766 - accuracy: 0.9775
Epoch 3/5
375/375 [==============================] - 5s 14ms/step - loss: 0.0510 - accuracy: 0.9849
Epoch 4/5
375/375 [==============================] - 5s 14ms/step - loss: 0.0361 - accuracy: 0.9896
Epoch 5/5
375/375 [==============================] - 1s 2ms/step - loss: 0.0496 - accuracy: 0.9843
Ep